# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [1]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
!pip install keras
!pip install tensorflow-cpu

以下のsolution関数のみ編集してください。

In [3]:
def solution(x_test_df, train_df):
    import numpy as np
    from skimage.transform import resize
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
    from keras.callbacks import EarlyStopping
    from sklearn.preprocessing import LabelEncoder
    from tensorflow.keras.utils import to_categorical
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau


    target_size = (64, 64)
    resized_images = []
    for img in train_df['waferMap']:
        resized_img = resize(img, target_size, anti_aliasing=True)
        resized_images.append(resized_img)

    X_train = np.array(resized_images).reshape(-1, 64, 64, 1)

    encoder = LabelEncoder()
    encoded_Y = encoder.fit_transform(train_df['failureType'])
    y_train = to_categorical(encoded_Y)

    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 1)))
    model.add(MaxPooling2D((5, 5)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(32, activation='relu'))
    
    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    # 学習率を自動的に減らすコールバックを作成します
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    
    model.fit(X_train, y_train, epochs=40, batch_size=256, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    resized_images_test = []
    for img in x_test_df['waferMap']:
        resized_img_test = resize(img, target_size, anti_aliasing=True)
        resized_images_test.append(resized_img_test)

    X_test = np.array(resized_images_test).reshape(-1, 64, 64, 1)
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)

    return pd.DataFrame({'failureType': encoder.inverse_transform(y_pred)}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [4]:
%%timeit -r 1 -n 1

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)

print(f"平均精度：{average_accuracy*100:.2f}%")

2023-11-04 05:42:26.373364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 05:42:36.601479: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 338661376 exceeds 10% of free system memory.


ValueError: Data cardinality is ambiguous:
  x sizes: 82681
  y sizes: 22967
Make sure all arrays contain the same number of samples.